In [1]:
from os.path import dirname, join as pjoin
import scipy.io as sio
import numpy as np

## RWG1 Geometry calculations - all Chapters
Uses the structure mesh file, e.g. platefine.mat, as an input.

Creates the RWG edge element for every inner edge of the structure. The total number of elements is EdgesTotal.

Outputs the following arrays:

Edge first node number          

    Edge_(1,1:EdgesTotal)
   
Edge second node number  

    Edge_(2,1:EdgesTotal)
    
Plus triangle number  

    TrianglePlus(1:EdgesTotal)
   
Minus triangle number           

    TriangleMinus(1:EdgesTotal)
   
Edge length                     

    EdgeLength(1:EdgesTotal)
   
Edge element indicator          
    
    EdgeIndicator(1:EdgesTotal)

Also outputs areas and midpoints of separate triangles:

Triangle area                   

    Area(1:TrianglesTotal)
   
Triangle center                 

    Center(1:TrianglesTotal)      
   
This script may handle surfaces with T-junctions including monopoles over various metal surfaces and certain metal meshes

Copyright 2002 AEMM. Revision 2002/03/09 Chapter 2


In [2]:
mat_fname = pjoin('mesh', 'strip2.mat')

print(mat_fname)

mat_contents = sio.loadmat(mat_fname)

mesh\strip2.mat


In [3]:
p = mat_contents['p']
t = mat_contents['t']

In [4]:
[s1,s2] = p.shape
if s1 == 2:
    print("Add third dimension")
    p = np.append(p,np.zeros((1,s2)),axis=0) 

In [5]:
TrianglesTotal=t.shape[1]
TrianglesTotal

244

Find areas of separate triangles

In [6]:
Area = np.zeros(TrianglesTotal)
Center = np.zeros((3,TrianglesTotal))
for m in range(TrianglesTotal):
        N = t[0:3,m] - 1
        #print(N)
        Vec1 = p[:,N[0]] - p[:,N[1]]
        Vec2 = p[:,N[2]] - p[:,N[1]]
        Area[m] = np.linalg.norm(np.cross(Vec1,Vec2))/2
        Center[:,m] = 1/3*sum(np.transpose(np.array(p[:,N])))

Find all edge elements "Edge_" with at least two adjacent triangles

In [7]:
n = 0
Edge_l = []
TrianglePlus = []
TriangleMinus = []
for m in range(TrianglesTotal):
    N = t[0:3,m] - 1 
    for k in range(m+1,TrianglesTotal):
        M = t[0:3,k] - 1      
        a = 1 - np.array([1 if i else 0 for i in [all(N-M[0]),all(N-M[1]),all(N-M[2])]])
        if(sum(a)==2): #triangles m and k have common edge
            Edge_l.append([M[i] for i,j in enumerate(a) if j]); 
            TrianglePlus.append(m);
            TriangleMinus.append(k); 
            n = n + 1;

In [8]:
Edge_ = np.transpose(np.array(Edge_l))
Edge__ = np.transpose(np.array([[i[1], i[0]] for i in Edge_l]))
#print(Edge_.shape)
#print(Edge__.shape)

EdgesTotal = len(Edge_l)
EdgesTotal

255

This block is only meaningful for T junctions. It leaves only two edge elements at a junction

In [9]:
Remove = []
for m in range(1,EdgesTotal):
    Edge_m = np.transpose(np.tile([np.array(Edge_)[:,m]],(EdgesTotal,1)))
    Edge_sup = list(Edge_  -Edge_m)
    Edge__sup = list(Edge__  -Edge_m)
    Ind1 = [0 if (Edge_sup[0][i] == 0) and (Edge_sup[1][i] == 0) else 1 for i in range(EdgesTotal)]
    Ind2 = [0 if (Edge__sup[0][i] == 0) and (Edge__sup[1][i] == 0) else 1 for i in range(EdgesTotal)]
    A_ = [i*j for i,j in zip(Ind1,Ind2)]
    A = [i for i,j in enumerate(A_) if j == 0]
    if(len(A)==3):    #three elements formally exist at a junction   
        print("There is a junction!! Stop!!")
        #Out=find(t(4,TrianglePlus(A))==t(4,TriangleMinus(A)));
        #Remove=[Remove A(Out)];

#np.array(list(Edge_).pop(Remove))
#TrianglePlus.pop(Remove)
#TriangleMinus.pop(Remove)
#EdgesTotal=len(Edge_)

EdgeIndicator = np.transpose(t[3,TrianglePlus] + t[3,TriangleMinus])

Find edges' length

In [10]:
EdgeLength = np.zeros(EdgesTotal)
for m in range(EdgesTotal):
    EdgeLength[m]=np.linalg.norm(p[:,Edge_[0,m]]-p[:,Edge_[1,m]])

In [11]:
Center

array([[ 0.00000000e+00,  6.66666667e-03,  6.66666667e-03,
         6.66666667e-03,  6.66666667e-03,  6.66666667e-03,
         6.66666667e-03,  3.33333333e-03,  3.33333333e-03,
         3.33333333e-03,  3.33333333e-03,  3.33333333e-03,
         3.33333333e-03,  3.33333333e-03,  3.33333333e-03,
         3.33333333e-03,  3.33333333e-03,  3.33333333e-03,
         3.33333333e-03,  3.33333333e-03,  3.33333333e-03,
         3.33333333e-03,  3.33333333e-03,  3.33333333e-03,
         3.33333333e-03,  3.33333333e-03,  3.33333333e-03,
         3.33333333e-03,  3.33333333e-03,  3.33333333e-03,
         3.33333333e-03,  3.33333333e-03,  3.33333333e-03,
         3.33333333e-03,  3.33333333e-03,  3.33333333e-03,
         3.33333333e-03,  3.33333333e-03,  3.33333333e-03,
         3.33333333e-03,  3.33333333e-03,  3.33333333e-03,
         3.33333333e-03,  3.33333333e-03,  3.33333333e-03,
         3.33333333e-03,  3.33333333e-03,  3.33333333e-03,
         3.33333333e-03,  3.33333333e-03,  3.33333333e-0

## RWG2: Geometry calculations - all Chapters
Uses the mesh file from RWG1, mesh1.mat, as an input.

Creates the following parameters of the RWG edge elements: 

Position vector rho_c_plus from the free vertex of the "plus" triangle to its center

                                   RHO_Plus(1:3,1:EdgesTotal)
                                   
Position vector rho_c_minus from the center of the "minus" triangle to its free vertex 

                                   RHO_Minus(1:3,1:EdgesTotal)

In addition to these parameters creates the following arrays for nine subtriangles (barycentric subdivision):

Midpoints of nine subtriangles

                                   Center_(1:3,1:9,1:TrianglesTotal)  
                                   
Position vectors rho_c_plus from the free vertex of the "plus" triangle to nine subtriangle midpoints

                                   RHO__Plus(1:3,1:9,1:EdgesTotal)
                                   
Position vectors rho_c_minus from nine subtriangle midpoints to the free vertex of the "minus" triangle

                                   RHO__Minus(1:3,1:9,1:EdgesTotal)

See Rao, Wilton, Glisson, IEEE Trans. Antennas and Propagation, vol. AP-30, No 3, pp. 409-418, 1982.

Copyright 2002 AEMM. Revision 2002/03/05 
Chapter 2

In [34]:
IMT=[]
for m in range(TrianglesTotal):
    n1 = t[0,m] - 1
    n2 = t[1,m] - 1
    n3 = t[2,m] - 1
    M = Center[:,m]
    r1 = p[:,n1]
    r2 = p[:,n2]
    r3 = p[:,n3]
    r12 = r2-r1
    r23 = r3-r2
    r13 = r3-r1
    C1 = r1 + (1/3)*r12
    C2 = r1+(2/3)*r12
    C3 = r2+(1/3)*r23
    C4 = r2+(2/3)*r23
    C5 = r1+(1/3)*r13
    C6 = r1+(2/3)*r13
    a1 = np.transpose([1/3*(C1+C5+r1)])
    a2 = np.transpose([1/3*(C1+C2+M)])
    a3 = np.transpose([1/3*(C2+C3+r2)])
    a4 = np.transpose([1/3*(C2+C3+M)])
    a5 = np.transpose([1/3*(C3+C4+M)])
    a6 = np.transpose([1/3*(C1+C5+M)])
    a7 = np.transpose([1/3*(C5+C6+M)])
    a8 = np.transpose([1/3*(C4+C6+M)])
    a9 = np.transpose([1/3*(C4+C6+r3)])
    if m == 0:
        Center_ = [np.concatenate((a1,a2,a3,a4,a5,a6,a7,a8,a9), axis=1)]
    else:
        Center_ = np.concatenate((Center_,[np.concatenate((a1,a2,a3,a4,a5,a6,a7,a8,a9), axis=1)]), axis = 0)
        
Center_ = Center_.transpose(1,2,0)


In [22]:
m = 0
n1 = t[0,m] - 1
n2 = t[1,m] - 1
n3 = t[2,m] - 1
M = Center[:,m]
r1 = p[:,n1]
r2 = p[:,n2]
r3 = p[:,n3]
r12 = r2-r1
r23 = r3-r2
r13 = r3-r1
C1 = r1 + (1/3)*r12
C2 = r1+(2/3)*r12
C3 = r2+(1/3)*r23
C4 = r2+(2/3)*r23
C5 = r1+(1/3)*r13
C6 = r1+(2/3)*r13
a1 = np.transpose([1/3*(C1+C5+r1)])
a2 = np.transpose([1/3*(C1+C2+M)])
a3 = np.transpose([1/3*(C2+C3+r2)])
a4 = np.transpose([1/3*(C2+C3+M)])
a5 = np.transpose([1/3*(C3+C4+M)])
a6 = np.transpose([1/3*(C1+C5+M)])
a7 = np.transpose([1/3*(C5+C6+M)])
a8 = np.transpose([1/3*(C4+C6+M)])
a9 = np.transpose([1/3*(C4+C6+r3)])
Center_0 = np.concatenate((a1,a2,a3,a4,a5,a6,a7,a8,a9), axis=1)

In [23]:
Center_0

array([[ 0.00666667,  0.        , -0.00666667, -0.00333333, -0.00333333,
         0.00333333,  0.00333333,  0.        ,  0.        ],
       [ 0.99927397,  0.99927397,  0.99927397,  0.99854794,  0.99709588,
         0.99854794,  0.99709588,  0.99636985,  0.9949178 ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ]])

In [21]:
Center_1

array([[0.00888889, 0.00888889, 0.00888889, 0.00777778, 0.00555556,
        0.00777778, 0.00555556, 0.00444444, 0.00222222],
       [0.99206517, 0.99515466, 0.99824414, 0.99648828, 0.99606605,
        0.9933988 , 0.99297657, 0.9943102 , 0.99388797],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ]])

In [ ]:
np.concatenate(([Center_0],[Center_1]), axis=2)

In [24]:
Center_0.shape

(3, 9)

In [25]:
Center_1.shape

(3, 9)

In [27]:
np.concatenate(([Center_0],[Center_1]), axis=0).shape

(2, 3, 9)

In [29]:
np.concatenate(([Center_0],[Center_1]), axis=0)

array([[[ 0.00666667,  0.        , -0.00666667, -0.00333333,
         -0.00333333,  0.00333333,  0.00333333,  0.        ,
          0.        ],
        [ 0.99927397,  0.99927397,  0.99927397,  0.99854794,
          0.99709588,  0.99854794,  0.99709588,  0.99636985,
          0.9949178 ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,
          0.        ]],

       [[ 0.00888889,  0.00888889,  0.00888889,  0.00777778,
          0.00555556,  0.00777778,  0.00555556,  0.00444444,
          0.00222222],
        [ 0.99206517,  0.99515466,  0.99824414,  0.99648828,
          0.99606605,  0.9933988 ,  0.99297657,  0.9943102 ,
          0.99388797],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,
          0.        ]]])

In [35]:
Center_.shape

(3, 9, 244)

In [38]:
Center_[:,:,0]

array([[ 0.00666667,  0.        , -0.00666667, -0.00333333, -0.00333333,
         0.00333333,  0.00333333,  0.        ,  0.        ],
       [ 0.99927397,  0.99927397,  0.99927397,  0.99854794,  0.99709588,
         0.99854794,  0.99709588,  0.99636985,  0.9949178 ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ]])